In [112]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=2, style="whitegrid", rc={'figure.figsize' : (15,7)})
import matplotlib.pyplot as plt
from scipy import special
plt.style.use('dark_background')

In [113]:
url='https://drive.google.com/file/d/1mk45XHSQiPtWl8SpgX8SWYoFIwtZuzi3/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

In [114]:
df

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franz�sisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21
...,...,...,...,...
5004,11/4/14,US-2014-168802,Jack O'Briant,18
5005,7/24/14,US-2014-169320,Lena Hernandez,171
5006,9/8/14,US-2014-169488,Allen Armold,57
5007,8/29/14,US-2014-169502,Matthew Grinstein,113


In [115]:
# смотрим временной период данных
pd.DataFrame({'':['start date', 'end_date'],' ': [df['order_date'].min(),df['order_date'].max()]})

,,
0,start date,1/1/14
1,end_date,9/9/14


In [49]:
df.dtypes

order_date     object
order_id       object
customer       object
grand_total     int64
dtype: object

In [116]:
df.order_date = pd.to_datetime(df.order_date)

C:\Users\mail\AppData\Local\Temp\ipykernel_10404\565821996.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.order_date = pd.to_datetime(df.order_date)


In [117]:
last_date = df.order_date.max()
last_date

Timestamp('2014-12-31 00:00:00')

In [118]:
df = df.groupby('customer').agg({'order_date'  : lambda x: (last_date - x.max()).days,
                            'order_id'    : lambda x: len(x),
                            'grand_total' : lambda x: x.sum()})
df.order_date = df.order_date.astype(int)
df.columns = ['recency','frequency', 'monetary']
#df.recency = df.recency.astype(int)

In [119]:
df

,recency,frequency,monetary
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249
...,...,...,...
Xylona Preis,43,11,2375
Yana Sorensen,9,8,6721
Yoseph Carroll,4,5,5455


In [120]:
quantiles = df.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


In [136]:
def r (value):
    if value <= quantiles.recency[0.25]:
        return 1
    elif value <= quantiles.recency[0.5]:
        return 2
    elif value <= quantiles.recency[0.75]:
        return 3
    else:
        return 4
    
def fm (value, metric):
    if value <= quantiles[metric][0.25]:
        return 4
    elif value <= quantiles[metric][0.5]:
        return 3
    elif value <= quantiles[metric][0.75]:
        return 2
    else:
        return 1

In [129]:
df_rfm= df

In [137]:
df_rfm['r']= df['recency'].apply(lambda x: r(x))
df_rfm['f']= df['frequency'].apply(lambda x: fm(x, 'frequency'))
df_rfm['m']= df['monetary'].apply(lambda x: fm(x, 'monetary'))

In [149]:
df_rfm['rfm'] = df_rfm.r.astype(str) + df_rfm.f.astype(str) + df_rfm.m.astype(str)
df_rfm.reset_index()[['customer','r','f', 'm', 'rfm']]

,customer,r,f,m,rfm
0,Aaron Bergman,4,4,4,444
1,Aaron Hawkins,1,2,3,123
2,Aaron Smayling,3,2,2,322
3,Adam Bellavance,2,2,1,221
4,Adam Hart,2,1,2,212
...,...,...,...,...,...
788,Xylona Preis,2,1,2,212
789,Yana Sorensen,1,2,1,121
790,Yoseph Carroll,1,4,1,141
791,Zuschuss Carroll,2,1,1,211


In [150]:
df_rfm.value_counts('rfm')

rfm
444    67
443    37
144    31
344    31
111    30
       ..
134     2
424     2
331     2
412     2
114     1
Name: count, Length: 62, dtype: int64